In [ ]:
# Install library yang diperlukan
!pip install fastapi uvicorn nest-asyncio pyngrok

# ─── Import Library ───────────────────────────────────────────
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
import nest_asyncio
from pyngrok import ngrok, conf
import uvicorn

# ─── Inisialisasi Aplikasi ───────────────────────────────────
app = FastAPI()

# ─── Autentikasi Ngrok (ganti jika perlu) ────────────────────
conf.get_default().auth_token = "2wD8BSIl0AyBNX70gQjHaeFQRms_2FV6E5GH9iT7cZDVVScQq"

# ─── Model Data Film ─────────────────────────────────────────
class Film(BaseModel):
    title: str
    director: str
    cast: List[str]
    synopsis: str

# ─── Data Awal: 3 Film Populer Lain ──────────────────────────
film_storage = [
    {
        "title": "Inception",
        "director": "Christopher Nolan",
        "cast": ["Leonardo DiCaprio", "Joseph Gordon-Levitt", "Ellen Page"],
        "synopsis": "Seorang pencuri yang memiliki kemampuan untuk masuk ke dalam mimpi orang lain ditugaskan untuk menanamkan ide ke dalam pikiran targetnya."
    },
    {
        "title": "Interstellar",
        "director": "Christopher Nolan",
        "cast": ["Matthew McConaughey", "Anne Hathaway", "Jessica Chastain"],
        "synopsis": "Sebuah tim penjelajah ruang angkasa melakukan perjalanan melalui wormhole untuk mencari planet baru yang layak huni bagi umat manusia."
    },
    {
        "title": "Parasite",
        "director": "Bong Joon-ho",
        "cast": ["Song Kang-ho", "Lee Sun-kyun", "Cho Yeo-jeong"],
        "synopsis": "Keluarga miskin di Korea menyusup ke kehidupan keluarga kaya, mengakibatkan serangkaian kejadian tak terduga yang penuh ketegangan dan satire sosial."
    },
]

# ─── Endpoint RESTful API ────────────────────────────────────
@app.get("/api/Movies")
def tampilkan_semua_film():
    return film_storage

@app.get("/api/Movies/{film_id}")
def ambil_film_berdasarkan_id(film_id: int):
    if 0 <= film_id < len(film_storage):
        return film_storage[film_id]
    return {"status": "gagal", "pesan": "Film tidak ditemukan"}

@app.post("/api/Movies")
def tambah_film(film: Film):
    film_storage.append(film.dict())
    return {"status": "berhasil", "pesan": "Film berhasil ditambahkan"}

@app.delete("/api/Movies/{film_id}")
def hapus_film(film_id: int):
    if 0 <= film_id < len(film_storage):
        data_dihapus = film_storage.pop(film_id)
        return {"status": "berhasil", "pesan": "Film dihapus", "film": data_dihapus}
    return {"status": "gagal", "pesan": "Film tidak ditemukan"}

# ─── Jalankan Server FastAPI ─────────────────────────────────
nest_asyncio.apply()
tunnel_url = ngrok.connect(8000)
print("🎥 Swagger UI:", tunnel_url.public_url + "/docs")
print("🎥 API Endpoint:", tunnel_url.public_url)

uvicorn.run(app, port=8000)